POC on Implementing Semantic Cache

In [ ]:
!pip install -U faiss-cpu sentence_transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 31.4 MB/s eta 0:00:00


In [ ]:
import faiss
import sqlite3
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from pprint import pprint


In [ ]:
import markdown
from IPython.display import Markdown

def print_gpt_markdown(markdown_text):
    display(Markdown(markdown_text))

In [ ]:
from google.colab import userdata
import requests

def make_prediction(data):
    url = "https://api-ares.traversaal.ai/live/predict"
    headers = {
        "x-api-key": userdata.get('ARES_KEY'),
        "content-type": "application/json"
    }

    payload = {"query": data}

    try:
        response = requests.post(url, json=payload, headers=headers)

        if response.status_code == 200:
            # The request was successful
            print("Request was successful.")
            # If the response contains JSON data, you can parse it using response.json()
            try:
                json_data = response.json()
                #print("Parsed JSON data:", json_data)
                return json_data
            except ValueError:
                print("No JSON data in the response.")
                return None
        else:
            # The request was not successful, handle the error
            print(f"Request failed with status code {response.status_code}.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None

# Example usage



In [ ]:
response=make_prediction(['Events happening in London this week. ']) #query should be inside square brackets

Request was successful.


In [ ]:
response

{'data': {'response_text': 'Here are some events happening in London this week (January 5-11, 2025):\n\n1. **New Museum Opening** - A new museum is launching this week, offering fresh exhibitions and experiences.\n   \n2. **Beer Festival** - Enjoy a variety of craft beers and local brews at a festival dedicated to beer enthusiasts.\n\n3. **Afternoon Tea Week** - Participate in special afternoon tea events across various venues, celebrating this quintessentially British tradition.\n\n4. **Friends TV Show Fan Event** - A special event for fans of the iconic TV show "Friends," featuring themed activities and memorabilia.\n\n5. **Formula 1 Racing Exhibition** - An exhibition showcasing the history and excitement of Formula 1 racing, perfect for motorsport fans.\n\nFor more details on specific events, you can check the following links:\n\n- [Eventbrite - Events this week in London](https://www.eventbrite.com/d/united-kingdom--london/events--this-week/)\n- [Londonist - Things To Do This Week

In [ ]:
print_gpt_markdown(response['data']['response_text'])

Here are some events happening in London this week (January 5-11, 2025):

1. **New Museum Opening** - A new museum is launching this week, offering fresh exhibitions and experiences.
   
2. **Beer Festival** - Enjoy a variety of craft beers and local brews at a festival dedicated to beer enthusiasts.

3. **Afternoon Tea Week** - Participate in special afternoon tea events across various venues, celebrating this quintessentially British tradition.

4. **Friends TV Show Fan Event** - A special event for fans of the iconic TV show "Friends," featuring themed activities and memorabilia.

5. **Formula 1 Racing Exhibition** - An exhibition showcasing the history and excitement of Formula 1 racing, perfect for motorsport fans.

For more details on specific events, you can check the following links:

- [Eventbrite - Events this week in London](https://www.eventbrite.com/d/united-kingdom--london/events--this-week/)
- [Londonist - Things To Do This Week](https://londonist.com/things-to-do-in-london-this-week)
- [Fever - Events & Things to Do in London this week](https://feverup.com/en/london/when/this-week?srsltid=AfmBOoqc6DSNVf_8rfx0_fPEnPXAQo6OtJwqX6sSOUv28V0c1zm-6eKZ) 

These resources will provide you with more information on specific locations and timings for each event.

In [ ]:
response['data']['web_url']

['https://www.visitlondon.com/things-to-do/whats-on/special-events/london-events-calendar',
 'https://www.timeout.com/london',
 'https://www.eventbrite.com/d/united-kingdom--london/events--this-week/',
 'https://londonist.com/things-to-do-in-london-this-week',
 'https://www.timeout.com/london/things-to-do-in-london-this-weekend',
 'https://www.eventbrite.com/d/united-kingdom--london/events--this-weekend/',
 'https://www.londontourism.ca/events',
 'https://londontheinside.com/whats-on-this-week/',
 'https://feverup.com/en/london/when/this-week?srsltid=AfmBOoqc6DSNVf_8rfx0_fPEnPXAQo6OtJwqX6sSOUv28V0c1zm-6eKZ',
 'https://www.visitlondon.com/things-to-do/whats-on']

Please replace with RAG/Agentic RAG

In [ ]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

class SemanticCaching:
    def __init__(self, json_file='cache.json'):
        # Faiss index with Euclidean distance
        self.index =faiss.IndexFlatL2(768)
        if self.index.is_trained:
            print('Index trained')

        # Sentence Transformer model
        self.encoder = SentenceTransformer('all-mpnet-base-v2')

        # Set Euclidean distance threshold
        self.euclidean_threshold = 0.3
        self.json_file = json_file
        self.load_cache()

    def load_cache(self):
        # Load cache from JSON file, creating an empty cache if the file is not found
        try:
            with open(self.json_file, 'r') as file:
                self.cache = json.load(file)
        except FileNotFoundError:
            self.cache = {'questions': [], 'embeddings': [], 'answers': [], 'response_text': []}

    def save_cache(self):
        # Save the cache to the JSON file
        with open(self.json_file, 'w') as file:
            json.dump(self.cache, file)

    def ask(self, question: str) -> str:
        # Method to retrieve an answer from the cache or generate a new one
        start_time = time.time()
        try:
            l = [question]
            embedding = self.encoder.encode(l)

            # Search for the nearest neighbor in the index
            D, I = self.index.search(embedding, 1)
           # print("printing the values of D and I below")
            #print(D)
            #print(I)
            #print(type(D))
            #print(type(I))

            if D[0] >= 0:
                if I[0][0] != -1 and D[0][0] <= self.euclidean_threshold:
                    row_id = int(I[0][0])
                    print(f'Found cache in row: {row_id} with score {1 - D[0][0]}') #score inversed to show similarity
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    print(f"Time taken: {elapsed_time} seconds")
                    return self.cache['response_text'][row_id]

            answer, response_text = self.generate_answer(question)

            self.cache['questions'].append(question)
            self.cache['embeddings'].append(embedding[0].tolist())
            self.cache['answers'].append(answer)
            self.cache['response_text'].append(response_text)

            self.index.add(embedding)
            self.save_cache()
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Time taken: {elapsed_time} seconds")

            return response_text
        except Exception as e:
            raise RuntimeError(f"Error during 'ask' method: {e}")

    def generate_answer(self, question: str) -> str:
        try:
            result = make_prediction([question])
            response_text = result['data']['response_text']
            return result, response_text
        except Exception as e:
            raise RuntimeError(f"Error during 'generate_answer' method: {e}")


In [ ]:
cache = SemanticCaching()



Index trained


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
question1 = "What is the capital of France?"
answer1 = cache.ask(question1)
print_gpt_markdown(answer1)

question2 = "Who is the CEO of Apple?"
answer2 = cache.ask(question2)
print_gpt_markdown(answer2)

question3 = "Who is the CEO of Facebook?"
answer3 = cache.ask(question3)
print_gpt_markdown(answer3)




Request was successful.
Time taken: 5.706909656524658 seconds


The capital of France is Paris. 

- Paris is the largest city in France, with an estimated population of approximately 2,102,650 residents as of January 2023.
- It is located along the Seine River in the north-central part of the country.
- Paris has been the capital of France since its liberation in 1944 and is one of the world's most important cultural and commercial centers.
- The city is often referred to as the "Fashion capital of the World" and is known for its rich history, art, and architecture. 

For more information, you can visit the following links:
- [Paris - Wikipedia](https://en.wikipedia.org/wiki/Paris)
- [Paris, France - Intercultural cities programme - The Council of Europe](https://www.coe.int/en/web/interculturalcities/paris)
- [Paris | Definition, Map, Population, Facts, & History - Britannica](https://www.britannica.com/place/Paris)

Request was successful.
Time taken: 3.836665391921997 seconds


The CEO of Apple is Tim Cook. He has held this position since August 2011, succeeding Steve Jobs. 

### Additional Information:
- **Employer**: Apple Inc. (1998–present)
- **Board Membership**: Tim Cook is also a board member of Nike, Inc.
- **Managerial Style**: Cook's managerial style is considered democratic, contrasting with Steve Jobs's more autocratic approach.
- **Net Worth**: As of 2023, Tim Cook's net worth is estimated to be $1.9 billion.

### Relevant Links:
- [Tim Cook - Wikipedia](https://en.wikipedia.org/wiki/Tim_Cook)
- [Tim Cook - Apple Leadership](https://www.apple.com/leadership/tim-cook/)
- [Tim Cook - Forbes](https://www.forbes.com/profile/tim-cook/)
- [Tim Cook: Apple CEO's Strengths, Reputation, and FAQs](https://www.investopedia.com/ask/answers/032515/what-tim-cooks-managerial-style.asp)

Request was successful.
Time taken: 3.4980788230895996 seconds


The CEO of Facebook is Mark Zuckerberg. 

- Mark Zuckerberg is the co-founder, chairman, and CEO of Meta Platforms, which was originally founded as Facebook in 2004. 
- He has held the position of CEO since the company's inception and continues to lead the organization as it evolves.
- In addition to his role at Meta, he is also the co-founder and co-CEO of the Chan Zuckerberg Initiative.

For more information, you can visit his LinkedIn profile: [Mark Zuckerberg - LinkedIn](https://www.linkedin.com/in/mark-zuckerberg-618bba58) or his Wikipedia page: [Mark Zuckerberg - Wikipedia](https://en.wikipedia.org/wiki/Mark_Zuckerberg).

In [ ]:
answer4 = cache.ask("What is the Capital of India")
print_gpt_markdown(answer4)

Request was successful.
Time taken: 9.541388273239136 seconds


The capital of India is New Delhi. 

- New Delhi is part of the National Capital Territory of Delhi (NCT).
- It serves as the seat of all three branches of the Government of India.
- Geographically, New Delhi is located in the north-central part of India, on the west bank of the Yamuna River. 

For more information, you can visit:
- [New Delhi - Wikipedia](https://en.wikipedia.org/wiki/New_Delhi)
- [New Delhi | History, Population, Map, & Facts - Britannica](https://www.britannica.com/place/New-Delhi)
- [What is the capital of India? - USA Today](https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/)

In [ ]:
answer4 = cache.ask("Can you tell me the Capital of India")
print_gpt_markdown(answer4)

Found cache in row: 3 with score 0.7601713836193085
Time taken: 0.01801156997680664 seconds


The capital of India is New Delhi. 

- New Delhi is part of the National Capital Territory of Delhi (NCT).
- It serves as the seat of all three branches of the Government of India.
- Geographically, New Delhi is located in the north-central part of India, on the west bank of the Yamuna River. 

For more information, you can visit:
- [New Delhi - Wikipedia](https://en.wikipedia.org/wiki/New_Delhi)
- [New Delhi | History, Population, Map, & Facts - Britannica](https://www.britannica.com/place/New-Delhi)
- [What is the capital of India? - USA Today](https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/)

In [ ]:
print_gpt_markdown(cache.ask('Who is the CEO of Meta?'))

Request was successful.
Time taken: 5.330321788787842 seconds


The CEO of Meta is Mark Zuckerberg. He is the founder, chairman, and CEO of the company, which he originally established as Facebook in 2004. Zuckerberg is responsible for setting the overall direction and product strategy for Meta.

### Additional Information:
- **Title**: Mark Zuckerberg, Founder, Chairman and Chief Executive Officer
- **Link**: [Mark Zuckerberg - Meta Executives](https://about.meta.com/media-gallery/executives/mark-zuckerberg/)
- **Other Roles**: In addition to his role at Meta, he is also a co-founder and co-CEO of the Chan Zuckerberg Initiative.
- **Recent Developments**: In 2023, Zuckerberg's leadership was highlighted during a "year of efficiency" for Meta, which saw significant stock performance improvements.
- **Compensation**: In 2023, he received a nominal salary of $1 but earned over $24 million in other forms of compensation.

For more details about Meta's executives, you can visit the [Executives - Meta](https://about.meta.com/media-gallery/executives/) page.

In [ ]:
print_gpt_markdown(cache.ask('Who is the current CEO of Google?'))

Request was successful.
Time taken: 3.505753755569458 seconds


The current CEO of Google is Sundar Pichai. He is also the CEO of Alphabet Inc., Google's parent company. 

- Sundar Pichai joined Google in 2004 and played a significant role in the development of key products such as Google Toolbar and Google Chrome, which has become the world's most popular internet browser.
- He serves on Alphabet's Board of Directors and is responsible for the day-to-day operations, product, and technology strategy for both Google and Alphabet.
- Under his leadership, Google has focused on developing innovative products and services, particularly in the realm of artificial intelligence.

For more information, you can refer to the following sources:
- [Sundar Pichai - Wikipedia](https://en.wikipedia.org/wiki/Sundar_Pichai)
- [Sundar Pichai | Google Blog](https://blog.google/authors/sundar-pichai/)
- [Sundar Pichai - CEO - Google - LinkedIn](https://www.linkedin.com/in/sundarpichai)

In [ ]:
print_gpt_markdown(cache.ask('CEO of Google?'))

Found cache in row: 4 with score 0.7289693653583527
Time taken: 0.021228313446044922 seconds


The current CEO of Google is Sundar Pichai. He is also the CEO of Alphabet Inc., Google's parent company. 

- Sundar Pichai joined Google in 2004 and played a significant role in the development of key products such as Google Toolbar and Google Chrome, which has become the world's most popular internet browser.
- He serves on Alphabet's Board of Directors and is responsible for the day-to-day operations, product, and technology strategy for both Google and Alphabet.
- Under his leadership, Google has focused on developing innovative products and services, particularly in the realm of artificial intelligence.

For more information, you can refer to the following sources:
- [Sundar Pichai - Wikipedia](https://en.wikipedia.org/wiki/Sundar_Pichai)
- [Sundar Pichai | Google Blog](https://blog.google/authors/sundar-pichai/)
- [Sundar Pichai - CEO - Google - LinkedIn](https://www.linkedin.com/in/sundarpichai)

In [ ]:
print_gpt_markdown(cache.ask('Best local food spots in Edinburgh for a couple?'))

Found cache in row: 7 with score 1.0
Time taken: 0.018563270568847656 seconds


The capital of India is New Delhi. 

- New Delhi is part of the National Capital Territory of Delhi (NCT).
- It serves as the seat of all three branches of the Government of India.
- Geographically, New Delhi is located in the north-central part of India, on the west bank of the Yamuna River. 

For more information, you can visit:
- [New Delhi - Wikipedia](https://en.wikipedia.org/wiki/New_Delhi)
- [New Delhi | History, Population, Map, & Facts - Britannica](https://www.britannica.com/place/New-Delhi)
- [What is the capital of India? - USA Today](https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/)

In [ ]:
print_gpt_markdown(cache.ask('Best local food spots in Edinburgh?'))

Found cache in row: 7 with score 0.8507776260375977
Time taken: 0.029055118560791016 seconds


The capital of India is New Delhi. 

- New Delhi is part of the National Capital Territory of Delhi (NCT).
- It serves as the seat of all three branches of the Government of India.
- Geographically, New Delhi is located in the north-central part of India, on the west bank of the Yamuna River. 

For more information, you can visit:
- [New Delhi - Wikipedia](https://en.wikipedia.org/wiki/New_Delhi)
- [New Delhi | History, Population, Map, & Facts - Britannica](https://www.britannica.com/place/New-Delhi)
- [What is the capital of India? - USA Today](https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/)

In [ ]:
print_gpt_markdown(cache.ask('Best local food spots in London?'))

Request was successful.
Time taken: 7.833227157592773 seconds


Here are some of the best local food spots in London:

1. **Clipstone**  
   - Location: Fitzrovia  
   - Description: A modern British restaurant known for its seasonal dishes.

2. **Barrafina**  
   - Location: Covent Garden  
   - Description: A popular tapas bar offering a vibrant atmosphere and authentic Spanish cuisine.

3. **The Quality Chop House**  
   - Location: Farringdon  
   - Description: A traditional British restaurant famous for its meat dishes and classic dining experience.

4. **Gökyüzü**  
   - Location: Harringay  
   - Description: Renowned for its Turkish cuisine, particularly its kebabs and meze.

5. **Casa Pastor**  
   - Location: King's Cross  
   - Description: A Mexican restaurant known for its tacos and vibrant flavors.

6. **Tayyabs**  
   - Location: Whitechapel  
   - Description: A beloved Pakistani restaurant famous for its grilled meats and curries.

7. **Oklava**  
   - Location: Shoreditch  
   - Description: A contemporary Turkish restaurant offering a unique twist on traditional dishes.

8. **Bright**  
   - Location: Hackney  
   - Description: A restaurant focusing on fresh, seasonal ingredients with a creative menu.

For more recommendations and details, you can visit the link: [Best local restaurants in London](https://www.visitlondon.com/things-to-do/food-and-drink/restaurant/local-restaurants). 

Additionally, here are some other resources for exploring local dining options:

- **An Eater's Guide to London**: This guide highlights interesting modern kitchens and Michelin-starred restaurants. [Read more here](https://london.eater.com/22586805/best-food-london-restaurants-where-to-eat).

- **The Good Food Guide**: Lists the best local restaurants in London, including Giulia in Shepherd's Bush and Home SW15 in Putney. [Explore the guide](https://www.thegoodfoodguide.co.uk/best-local-restaurant/best-local-restaurant-award-winners-2024/london).

- **Hotels.com**: Offers a list of the 10 best local restaurants, including The Wolseley and Riding House Cafe. [Check it out](https://www.hotels.com/go/england/uk-london-local-restaurants).

These resources will help you discover a variety of dining experiences across London.

In [ ]:
print_gpt_markdown(cache.ask('Best breakfast spots in Sacramento?'))

Request was successful.
Time taken: 5.5322630405426025 seconds


Here are some of the best breakfast spots in Sacramento:

1. **The Morning Fork**  
   - A popular choice known for its delicious breakfast options.

2. **Toasted Rooster Cafe**  
   - Offers a cozy atmosphere with a variety of breakfast dishes.

3. **Bacon & Butter**  
   - Renowned for its fresh ingredients and creative menu.

4. **Orphan Breakfast House**  
   - A favorite for its unique breakfast offerings and vibrant ambiance.

5. **Four Sisters Cafe**  
   - Known for its homey feel and hearty breakfast plates.

6. **Hot Off The Griddle**  
   - A great spot for classic breakfast favorites.

7. **Tower Cafe**  
   - Famous for its eclectic menu and beautiful outdoor seating.

8. **Grange Restaurant & Bar**  
   - Offers a farm-to-fork experience with a focus on local ingredients.

9. **Fox & Goose Public House**  
   - A British-style pub that serves a fantastic breakfast.

10. **Ettore's Bakery & Cafe**  
    - Known for its pastries and breakfast items in a charming setting.

11. **Hungry Hollow Cafe**  
    - A local favorite, especially busy on weekends, known for fast service and great food.

12. **FlapJacks Diner**  
    - A casual diner with a variety of breakfast options.

13. **Nopalitos Southwestern Cafe**  
    - Offers a unique southwestern twist on traditional breakfast dishes.

14. **Bella Bru Cafe**  
    - A popular spot for brunch with a diverse menu.

15. **Crepeville**  
    - Specializes in crepes and other breakfast delights.

For more details, you can explore reviews and menus on platforms like Yelp, TripAdvisor, and local food blogs.